# Imports

In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd
from custom_modules.helper_functions import * 

# Read Data

In [2]:
# Read spot and forward bid, mid and ask rates.
df_sm = pd.read_csv("..\\Data\\Raw Data\\Currencies\\spot_mid.csv",
                    parse_dates=[0])
df_sb = pd.read_csv("..\\Data\\Raw Data\\Currencies\\spot_bid.csv",
                    parse_dates=[0])
df_sa = pd.read_csv("..\\Data\\Raw Data\\Currencies\\spot_ask.csv",
                    parse_dates=[0])

df_fm = pd.read_csv("..\\Data\\Raw Data\\Currencies\\forward_mid.csv",
                    parse_dates=[0])
df_fb = pd.read_csv("..\\Data\\Raw Data\\Currencies\\forward_bid.csv",
                    parse_dates=[0])
df_fa = pd.read_csv("..\\Data\\Raw Data\\Currencies\\forward_ask.csv",
                    parse_dates=[0])

# Read information on currency pair codes.
df_info = pd.read_csv("..\\Data\\Raw Data\\Currencies\\currency_info.csv")

C:\Users\uqjhear1\miniconda3\envs\phd\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '31/01/1976' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\uqjhear1\miniconda3\envs\phd\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '27/02/1976' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\uqjhear1\miniconda3\envs\phd\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '31/03/1976' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\uqjhear1\miniconda3\envs\phd\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '30/04/1976' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  re

# Data Aggregation Function

In [3]:
def build_data(data_in, terms, use_points=[], points_denom=[],
               points_source_rate=[]):
    """
    This function takes a sequence of data arrays and stitches them
    together into a single array with priority given to the arrays that
    come later in the sequence.

    Parameters
    ----------
    data_in : array_like or sequence of array_like
        The elements of the sequence are assumed to be aligned so that
        the final element of all of the arrays corresponds to an
        observation that occurs on the same date across each array. The
        resulting array, which will have the same length as the longest
        array in the sequence, will have elements given by the
        corresponding element from the array with the highest priority
        that had a non-missing elements in that position. Arrays later
        in the sequence have higher priority, so that the resulting
        array will have final elements equal exactly to the final array
        in the sequence, extended by elements in earlier positions that
        are taken by any longer arrays moving successively in that
        manner from the right-most array to the left-most. If a single
        array is given, that array is returned after checking and
        correcting for forward points and the terms of the series.
    terms : string or sequence of string
        A sequence of strings equal to either "american" or "european".
        Each sequence element provides the terms of the data series with
        the same index in the data_in sequence. The length of this
        parameter must match the length of data_in, where the length of
        a non-sequence is considered to be one.
    use_points : int or sequence of int, default []
        If an int is given, then that int is used as an index to point
        to one array from the data_in sequence that is given in forward
        points rather than as a direct rate. If a list of ints is given,
        each int in the list is an index pointing to input data that is
        given in forward points.
    points_denom : int or sequence of int, default []
        For each array given in forward points, this parameter gives the
        denominator of the scale of those points, where the order is
        determined by the order of the use_points indices. So, the first
        points_denom gives the denominator of the first array given in
        forward points that is declared in the use_points sequence. The
        length of this parameter must match the length of use_points,
        where the length of a non-sequence is considered to be one.
    points_source_rate : string or sequence of string, default []
        The source spot rates to which forward point series must be
        added to calculate direct rates, with the same order as
        use_points. The length of this parameter must match the length
        of use_points, where the length of a non-sequence is considered
        to be one.

    Returns
    -------
    output : array_like
        The aggregated array of datapoints.
    """

    # --- SCRUB INPUTS ---
    # Verify data_in
    if isinstance(data_in, (pd.Series, np.ndarray)):
        data_in = [data_in]
    elif isinstance(data_in, (tuple, list)):
        for i in data_in:
            if not isinstance(i, (pd.Series, np.ndarray)):
                raise TypeError("Elements of data_in must be of type Series "
                                "or array, not {}".format(type(i).__name__))
    else:
        raise TypeError("data_in must be of type Series, array, list or "
                        "tuple, not {}".format(type(run_data).__name__))
    # Verify terms
    if isinstance(terms, str):
        if terms.lower() in ["american", "european"]:
            terms = [terms]
        else:
            raise ValueError("terms must be either 'american' or 'european', "
                             " not {}".format(i))
    elif isinstance(terms, (tuple, list)):
        for i in terms:
            if i.lower() not in ["american", "european"]:
                raise ValueError("Elements of terms must be either 'american'"
                                 "or 'european', not {}".format(i))
    else:
        raise TypeError("terms must be of type string, list or tuple, "
                        "not {}".format(type(terms).__name__))

    if len(data_in) != len(terms):
        raise ValueError("Size mismatch between data_in and terms.")

    # Copy contents of data_in
    run_data = [i.copy() for i in data_in]

    # Verify use_points
    try:
        if int(use_points) == use_points:
            use_points = [use_points]
        else:
            raise ValueError("use_points cannot be interpreted as an integer.")
    except TypeError:
        if isinstance(use_points,  (tuple, list)):
            for i in use_points:
                try:
                    if int(i) != i:
                        raise ValueError("use_points contains at least one"
                                         "element that can't be interpreted"
                                         "as an integer.")
                except TypeError:
                    raise TypeError("Elements of use_points must be of type"
                                    "int, not {}".format(type(i).__name__))
        else:
            raise TypeError("use_points must be of type int, list or tuple, "
                            "not {}".format(type(use_points).__name__))

    # Verify points_source_rate
    if isinstance(points_source_rate,  (pd.Series, np.ndarray)):
        points_source_rate = [points_source_rate]
    elif isinstance(points_source_rate, (tuple, list)):
        for i in points_source_rate:
            if not isinstance(i, (pd.Series, np.ndarray)):
                raise TypeError("Elements of use_points must be of type "
                                "Series or array, "
                                "not {}".format(type(i).__name__))
    else:
        raise TypeError("use_points must be of type Series, array, list or "
                        "tuple, not {}".format(type(i).__name__))

    # Verify points_denom
    try:
        if int(points_denom) == points_denom:
            points_denom = [points_denom]
        else:
            raise ValueError("point_denom cannot be interpreted as an "
                             "integer.")
    except TypeError:
        if isinstance(points_denom, (tuple, list)):
            for i in points_denom:
                try:
                    if int(i) != i:
                        raise ValueError("points_denom contains at least one "
                                         "element that can't be interpreted "
                                         "as an integer.")
                except TypeError:
                    raise TypeError("Elements of points_denom must be of type "
                                    "int, not {}".format(type(i).__name__))
        else:
            raise TypeError("points_denom must be of type int, list or tuple, "
                            "not {}".format(type(use_points).__name__))

    # Verify matching forward point list lengths
    if len(use_points) > 0 and (len(use_points) != len(points_source_rate)):
        raise ValueError("Size mismatch between use_points and "
                         "points_source_rate.")

    if len(use_points) > 0 and (len(use_points) != len(points_denom)):
        raise ValueError("Size mismatch between use_points and points_denom.")
   
    # --- PROCESS DATA ---
    # Add forward points
    for i in range(len(use_points)):
        # If they are indicated, match the size of the forward points
        # series with the size of the source spot series.
        if len(run_data[use_points[i]]) != len(points_source_rate[i]):
            raise ValueError("Size mismatch between a declared forward-points "
                             "series (index: {}) and the given source spot "
                             "rate series.".format(use_points[i]))
        else:
            # Add the forward points to the source spot rate using the
            # provided denominator.
            run_data[use_points[i]] = (run_data[use_points[i]]/points_denom[i]
                                       + points_source_rate[i])

    # Set all series to European terms by flipping those given in American
    # terms.
    for i in range(len(run_data)):
        if terms[i].lower() == "american":
            run_data[i] = 1/run_data[i]
            
    # Calculate the aggregated series by first setting the output
    # variable as the first array in the input sequence, then
    # progressively overwriting the output variable with all non-nan
    # observations of each successive array in the list.
    output = run_data[0]
    for i in range(1, len(run_data)):
        output = np.where(run_data[i].isnull(), output, run_data[i])

    return output

# Series Information

Declare a number of dictionaries containing information about each currency series.

In [4]:
# Group info dataframe by currency code to collect lists of info about
# each final currency series
curs = df_info.groupby("cur_code")

# Initialise empty dictionaries
terms = dict()
spot_symbols = dict()
forward_symbols = dict()
forward_pt_idx = dict()
forward_pt_denoms = dict()

# For each currency, add the following to each dictionary:
for name, group in curs:
    # Whether each component series is American or European terms
    terms[name] = list(group.terms.values)
    
    # The spot symbols for each component series
    spot_symbols[name] = list(group.symbol_s.values)
    
    # The forward symbols for each component series
    forward_symbols[name] = list(group.symbol_f.values)
    
    # The index values of the forward component series that are given in
    # forward points
    forward_pt_idx[name] = list(np.where(group.f_denom.notnull())[0])
    
    # The denominators of those forward points series
    forward_pt_denoms[name] = list(group.f_denom.dropna())

In [5]:
df_sm_combined = df_sm[["date"]].copy()
df_sb_combined = df_sb[["date"]].copy()
df_sa_combined = df_sa[["date"]].copy()
df_fm_combined = df_fm[["date"]].copy()
df_fb_combined = df_fb[["date"]].copy()
df_fa_combined = df_fa[["date"]].copy()

for i in spot_symbols:
    sm_in = df_sm[spot_symbols[i]].copy()
    sb_in = df_sb[spot_symbols[i]].copy()
    sa_in = df_sa[spot_symbols[i]].copy()

    fm_in = df_fm[forward_symbols[i]].copy()
    fb_in = df_fb[forward_symbols[i]].copy()
    fa_in = df_fa[forward_symbols[i]].copy()

    for j in range(len(terms[i])):
        if terms[i][j] == "american":
            # Flip the bid and ask series. When these series are entered
            # into the build_data function below, it will input that the
            # series is in american terms, and will be flipping the
            # correct series so that the output is either a bid or an
            # ask as desired (the inverse of a bid is an ask and vice
            # versa)
            temp_series_s = sb_in[spot_symbols[i][j]].copy()
            sb_in[spot_symbols[i][j]] = sa_in[spot_symbols[i][j]]
            sa_in[spot_symbols[i][j]] = temp_series_s

            temp_series_f = fb_in[forward_symbols[i][j]].copy()
            fb_in[forward_symbols[i][j]] = fa_in[forward_symbols[i][j]]
            fa_in[forward_symbols[i][j]] = temp_series_f

    df_sm_combined[i] = build_data([sm_in[k] for k in sm_in.columns], terms[i])
    df_sb_combined[i] = build_data([sb_in[k] for k in sb_in.columns], terms[i])
    df_sa_combined[i] = build_data([sa_in[k] for k in sa_in.columns], terms[i])

    df_fm_combined[i] = build_data([fm_in[k] for k in fm_in.columns], terms[i],
                                   forward_pt_idx[i], forward_pt_denoms[i],
                                   [sm_in[spot_symbols[i][m]] for m
                                    in forward_pt_idx[i]])

    df_fb_combined[i] = build_data([fb_in[k] for k in fb_in.columns], terms[i],
                                   forward_pt_idx[i], forward_pt_denoms[i],
                                   [sb_in[spot_symbols[i][m]] for m
                                    in forward_pt_idx[i]])

    df_fa_combined[i] = build_data([fa_in[k] for k in fa_in.columns], terms[i],
                                   forward_pt_idx[i], forward_pt_denoms[i],
                                   [sa_in[spot_symbols[i][m]] for m
                                    in forward_pt_idx[i]])

In [6]:
# Melt each DataFrame so they can be merged together
df_sb_melt = df_sb_combined.melt(id_vars="date", var_name="curr",
                                 value_name="spot_bid")
df_sm_melt = df_sm_combined.melt(id_vars="date", var_name="curr",
                                 value_name="spot_mid")
df_sa_melt = df_sa_combined.melt(id_vars="date", var_name="curr",
                                 value_name="spot_ask")
df_fb_melt = df_fb_combined.melt(id_vars="date", var_name="curr",
                                 value_name="for_bid")
df_fm_melt = df_fm_combined.melt(id_vars="date", var_name="curr",
                                 value_name="for_mid")
df_fa_melt = df_fa_combined.melt(id_vars="date", var_name="curr",
                                 value_name="for_ask")


# Merge and format resultant DataFrame
df_curr = (pd.merge(df_sb_melt, df_sm_melt, on=["date", "curr"])
             .merge(df_sa_melt, on=["date", "curr"])
             .merge(df_fb_melt, on=["date", "curr"])
             .merge(df_fm_melt, on=["date", "curr"])
             .merge(df_fa_melt, on=["date", "curr"])
             .dropna(subset=["spot_bid", "spot_mid", "spot_ask", "for_bid",
                             "for_mid", "for_ask"])
             .loc[:, ["curr", "date", "spot_bid", "spot_mid", "spot_ask",
                      "for_bid", "for_mid", "for_ask"]]
             .sort_values(by=["curr", "date"]))

# Align Dates to End of Month

In [7]:
df_curr.date = df_curr.date + MonthEnd(0)

# Delete Bad Observations

Delete known deviations from CIP, as well as the Euro components after they joined the Eurozone. Remove outter observations completely, but replace inner observations with null.

In [8]:
df_curr = df_curr.copy()[
    ~((df_curr.curr == "ATS") & (df_curr.date >= "1999-01-01")) &
    ~((df_curr.curr == "BEF") & (df_curr.date >= "1999-01-01")) &
    ~((df_curr.curr == "FIM") & (df_curr.date >= "1999-01-01")) &
    ~((df_curr.curr == "FRF") & (df_curr.date >= "1999-01-01")) &
    ~((df_curr.curr == "DEM") & (df_curr.date >= "1999-01-01")) &
    ~((df_curr.curr == "IEP") & (df_curr.date >= "1999-01-01")) &
    ~((df_curr.curr == "ITL") & (df_curr.date >= "1999-01-01")) &
    ~((df_curr.curr == "NGL") & (df_curr.date >= "1999-01-01")) &
    ~((df_curr.curr == "PTE") & (df_curr.date >= "1999-01-01")) &
    ~((df_curr.curr == "ESP") & (df_curr.date >= "1999-01-01")) &
    ~((df_curr.curr == "GRD") & (df_curr.date >= "2001-01-01"))
]

In [9]:
df_curr.loc[
    ((df_curr.curr == "ZAR") &
     (df_curr.date >= "1985-07-31") & (df_curr.date <= "1985-08-31")) |
    ((df_curr.curr == "MYR") &
     (df_curr.date >= "1998-08-31") & (df_curr.date <= "2005-06-30")) |
    ((df_curr.curr == "IDR") &
     (df_curr.date >= "2000-12-31") & (df_curr.date <= "2007-05-31")) |
    ((df_curr.curr == "TRY") &
     (df_curr.date >= "2000-10-31") & (df_curr.date <= "2001-11-30")) |
    ((df_curr.curr == "AED") &
     (df_curr.date >= "2006-06-30") & (df_curr.date <= "2006-11-30")),
    "spot_bid":"for_ask"
] = np.nan

# Create LRV11 Sample

Recreate the data subset that most closely matches the data from LRV 2011.

In [10]:
# Restrict to the currencies used in LRV 2011.
lrv_currs = df_curr.curr.isin(["AUD", "ATS", "BEF", "CAD", "HKD", "CZK", "DKK",
                              "EUR", "FIM", "FRF", "DEM", "GRD", "HUF", "INR",
                              "IDR", "IEP", "ITL", "JPY", "KWD", "MYR", "MXN",
                              "NLG", "NZD", "NOK", "PHP", "PLN", "PTE", "SAR",
                              "SGD", "ZAR", "KOW", "KRW", "ESP", "SEK", "CHF",
                              "TWD", "THB", "GBP"])

# Restrict to the dates used in LRV 2011
df_curr_lrv11 = df_curr.copy()[lrv_currs & (df_curr.date <= "2009-12-31")]

# Save Data

In [11]:
df_curr.to_csv("..\\Data\\Refined Data\\Currencies\\currencies.csv",
               index=False)

In [12]:
df_curr_lrv11.to_csv("..\\Data\\Refined Data\\Currencies\\"
                     "currencies_lrv11.csv", index=False)